In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Khoa_luan_tot_nghiep/HPSG-Parser/HPSG-Neural-Parser-master

/content/drive/.shortcut-targets-by-id/1DP82lkt284bhnm7X9kA_LML9IgUxQIL9/Khoa_luan_tot_nghiep/HPSG-Parser/HPSG-Neural-Parser-master


In [3]:
!pip3 install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install allennlp==0.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 7.5 MB/s 
     |████████████████████████████████| 125 kB 53.7 MB/s 
     |████████████████████████████████| 123 kB 54.0 MB/s 
     |████████████████████████████████| 5.8 MB 49.5 MB/s 
     |████████████████████████████████| 235 kB 53.9 MB/s 
     |████████████████████████████████| 158 kB 47.3 MB/s 
     |████████████████████████████████| 51 kB 744 kB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 
     |████████████████████████████████| 592 kB 47.0 MB/s 
     |████████████████████████████████| 132 kB 59.7 MB/s 
     |████████████████████████████████| 30.8 MB 40.7 MB/s 
     |████████████████████████████████| 48 kB 6.2 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 1.2 MB 45.6 MB/s 
     |████████████████████████████████| 251 kB 57.0 MB/s 
     |████████████████████████████████|

In [ ]:
!pip install pytorch-transformers

In [ ]:
!pip install cython

In [ ]:
!chmod 755 -R EVALB/evalb

In [ ]:
%cd /content/drive/My Drive/Khoa_luan_tot_nghiep/HPSG-Parser/HPSG-Neural-Parser-master/EVALB

In [ ]:
!apt-get install -qq gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 
!ln -s /usr/bin/g++-5 

!sudo apt-get update
!sudo apt-get upgrade

In [ ]:
!make

In [ ]:
%cd /content/drive/My Drive/Khoa_luan_tot_nghiep/HPSG-Parser/HPSG-Neural-Parser-master

In [ ]:
!sh run_single.sh

In [ ]:
!sh run_single.sh

In [ ]:
#test
!sh run_single.sh

In [ ]:
!sh parse.sh

In [ ]:
!sh test.sh

In [ ]:
!sh test.sh

In [ ]:
#test2
!sh run_single.sh

In [ ]:
#test 3, const-lambda = 0.8, default =0.5
!sh run_single.sh

In [ ]:
#test 4, ensemble, lada = 0.5
!sh run_single_ensemble.sh

In [ ]:
#test5 change num_layer
!sh run_single.sh

In [ ]:
import os.path
import tempfile

In [ ]:
evalb_dir = '/content/drive/My Drive/Khoa_luan_tot_nghiep/HPSG-Parser/HPSG-Neural-Parser-master/EVALB'

In [ ]:
evalb_program_path = os.path.join(evalb_dir, "evalb")

In [ ]:
assert os.path.exists(evalb_program_path)

In [ ]:
evalb_param_path = os.path.join(evalb_dir, "COLLINS.prm")

In [ ]:
import subprocess
temp_dir = tempfile.TemporaryDirectory(prefix="evalb-")
gold_path = os.path.join(temp_dir.name, "gold.txt")
predicted_path = os.path.join(temp_dir.name, "predicted.txt")
output_path = os.path.join(temp_dir.name, "output.txt")

In [ ]:
import subprocess
temp_dir = evalb_dir
gold_path = os.path.join(temp_dir, "gold.txt")
predicted_path = os.path.join(temp_dir, "predicted.txt")
output_path = os.path.join(temp_dir, "output.txt")

In [ ]:
command = "{} -p {} {} {} > {}".format(
        evalb_program_path,
        evalb_param_path,
        gold_path,
        predicted_path,
        output_path,
    )
result = subprocess.run(command, shell=True)
result.check_returncode()
print(result.returncode)
print(result)

In [ ]:
def tokenize_vi(input):
    
    new_token = []
    mode = 0
    tmp = []

    for i in range(len(input)):
        if(input[i] == '(' and mode == 0):
            mode = 1
            new_token.append(input[i])

        elif(input[i] == ')' and mode == 1):

            new_token.append(tmp[0])
            vn_token = ''
            for j in range(1, len(tmp) - 1):
                vn_token = vn_token + tmp[j] + '_'
            vn_token = vn_token + tmp[len(tmp) - 1]
            print(vn_token)
            new_token.append(vn_token)
            new_token.append(input[i])
            tmp = []
            mode = 0

        elif(input[i] == '(' and mode == 1):
            for j in range(0, len(tmp)):
                new_token.append(tmp[j])
            new_token.append(input[i])
            tmp = []
        elif(mode == 1):
            tmp.append(input[i])
        elif(mode == 0):
            new_token.append(input[i])
    return new_token

In [ ]:
def token_to_string(token):
  res = ''
  tmp = []
  mode = 0 #0 no open bracket, #1 yes open bracket
  for i in range(0, len(token)):
    if(token[i] == '(' and mode == 0):
      res = res + ' ' + token[i]
      mode = 1
    elif(token[i] == '(' and mode == 1):
      for tok in tmp:
        res = res + tok
      res = res + ' ' + token[i]
      tmp = []
    elif(token[i] == ')' and mode == 1):
      bracket_content = ''
      for tok in tmp:
        bracket_content = bracket_content + tok + ' '
      bracket_content = bracket_content.rstrip(' ')
      res = res + bracket_content + token[i]
      mode = 0
      tmp = []
    elif(token[i] == ')' and mode == 0):
      res = res  + token[i]
    else:
      tmp.append(token[i])
  res = res.lstrip(' ')
  return res

In [ ]:
input = '(TOP (S (NP (N-H Cuộc đời) (PP (E-H dưới) (NP (N-H vành) (N mũ) (NP (N-H thám tử))))) (. .)))'
sentence_tokens = input.replace("(", " ( ").replace(")", " ) ").split()
new_token = tokenize_vi(sentence_tokens)
print(new_token)

In [ ]:
new_line = token_to_string(new_token)
print(new_line)
print(new_line[1])

In [ ]:
class FScore(object):
    def __init__(self, recall, precision, fscore):
        self.recall = recall
        self.precision = precision
        self.fscore = fscore

    def __str__(self):
        return "(Recall={:.2f}, Precision={:.2f}, FScore={:.2f})".format(
            self.recall, self.precision, self.fscore)

In [ ]:
import math
import os.path
import re
fscore = FScore(math.nan, math.nan, math.nan)
with open(output_path) as infile:
  for line in infile:
      match = re.match(r"Bracketing Recall\s+=\s+(\d+\.\d+)", line)
      if match:
          fscore.recall = float(match.group(1))
      match = re.match(r"Bracketing Precision\s+=\s+(\d+\.\d+)", line)
      if match:
          fscore.precision = float(match.group(1))
      match = re.match(r"Bracketing FMeasure\s+=\s+(\d+\.\d+)", line)
      if match:
          fscore.fscore = float(match.group(1))
          break


In [ ]:
fscore = FScore(math.nan, math.nan, math.nan)

In [ ]:
%cd /content/drive/My Drive/Khoa_luan_tot_nghiep/HPSG-Parser/HPSG-Neural-Parser-master/EVALB

In [ ]:
import subprocess

evalb_program_path = 'evalb'
evalb_param_path = 'COLLINS.prm'
gold_path = 'gold.txt'
predicted_path = 'predicted.txt'
output_path = 'output.txt'
command = "{} -p {} {} {}  > {}".format(
    evalb_program_path,
    evalb_param_path,
    gold_path,
    predicted_path,
    output_path,
)

output = subprocess.run(command, shell=True, capture_output=True)
print(output)


In [ ]:
! ./evalb -p COLLINS.prm gold.txt predicted.txt 

In [ ]:
#ensemble
!sh test_ensemble.sh

In [ ]:
#ensemble1
!sh test_ensemble.sh

In [ ]:
#test 7, different word embedding
!sh run_single.sh

In [ ]:
#test 7, different word embedding
!sh run_single.sh

In [ ]:
#test 8
!sh run_bert.sh

In [ ]:
!sh test.sh

In [ ]:
#test 9, data fixed
!sh run_single.sh

In [ ]:
#continue of step 9
!sh run_single.sh

In [ ]:
#test 10: run 10 times to get average (for run8th folder to run17th)
!sh run_single.sh

In [ ]:
#test 11 test with newest data
!sh run_single.sh

In [ ]:
#test 11 10 run average(13th-23th)
!sh run_single.sh

Traceback (most recent call last):
  File "src_joint/main.py", line 7, in <module>
    import torch
  File "/usr/local/lib/python3.7/dist-packages/torch/__init__.py", line 751, in <module>
    from .functional import *  # noqa: F403
  File "/usr/local/lib/python3.7/dist-packages/torch/functional.py", line 8, in <module>
    import torch.nn.functional as F
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/__init__.py", line 1, in <module>
    from .modules import *  # noqa: F403
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/__init__.py", line 2, in <module>
    from .linear import Identity, Linear, Bilinear, LazyLinear
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/linear.py", line 6, in <module>
    from .. import functional as F
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py", line 3712, in <module>
    def interpolate(input: Tensor, size: Optional[int] = None, scale_factor: Optional[List[float]] = None, mode: str = 'near

In [ ]:
#test 12 with VnData10
!sh run_single.sh

Manual seed for pytorch: 1551707826
Hyperparameters:
attention_dropout 0.2
bert_do_lower_case True
bert_model 'bert-large-uncased'
bert_transliterate ''
char_lstm_input_dropout 0.2
clip_grad_norm 0.0
const_lada 0.5
d_biaffine 1024
d_char_emb 64
d_ff 2048
d_kv 64
d_label_hidden 250
d_model 1024
dataset 'ptb'
elmo_dropout 0.5
embedding_dropout 0.2
embedding_path 'data/word2vec_vi_words_100dims.txt.gz'
embedding_type 'sskip'
learning_rate 0.0008
learning_rate_warmup_steps 40
max_len_dev 0
max_len_train 0
model_name 'joint_single'
morpho_emb_dropout 0.2
num_heads 8
num_layers 12
pad_left False
partitioned True
punctuation ".``'':,"
relu_dropout 0.2
residual_dropout 0.2
sentence_max_len 300
step_decay True
step_decay_factor 0.5
step_decay_patience 5
tag_emb_dropout 0.2
timing_dropout 0.0
use_bert False
use_bert_only False
use_cat True
use_chars_lstm True
use_elmo False
use_tags True
use_words True
use_xlnet False
word_emb_dropout 0.4
xlnet_do_lower_case False
xlnet_model 'xlnet-large-cased'

In [ ]:
#test 13 no embedding
!sh run_single.sh

Manual seed for pytorch: 1601194947
Hyperparameters:
attention_dropout 0.2
bert_do_lower_case True
bert_model 'bert-large-uncased'
bert_transliterate ''
char_lstm_input_dropout 0.2
clip_grad_norm 0.0
const_lada 0.5
d_biaffine 1024
d_char_emb 64
d_ff 2048
d_kv 64
d_label_hidden 250
d_model 1024
dataset 'ptb'
elmo_dropout 0.5
embedding_dropout 0.2
embedding_path 'data/word2vec_vi_words_100dims.txt.gz'
embedding_type 'random'
learning_rate 0.0008
learning_rate_warmup_steps 40
max_len_dev 0
max_len_train 0
model_name 'joint_single'
morpho_emb_dropout 0.2
num_heads 8
num_layers 12
pad_left False
partitioned True
punctuation ['.', '`', "'", ':', ',', '"', '...', '/', ';', '-']
relu_dropout 0.2
residual_dropout 0.2
sentence_max_len 300
step_decay True
step_decay_factor 0.5
step_decay_patience 5
tag_emb_dropout 0.2
timing_dropout 0.0
use_bert False
use_bert_only False
use_cat True
use_chars_lstm True
use_elmo False
use_tags True
use_words True
use_xlnet False
word_emb_dropout 0.4
xlnet_do_lowe

In [ ]:
#test 14 with VnData11
!sh run_single.sh

Manual seed for pytorch: 1089484352
Hyperparameters:
attention_dropout 0.2
bert_do_lower_case True
bert_model 'bert-large-uncased'
bert_transliterate ''
char_lstm_input_dropout 0.2
clip_grad_norm 0.0
const_lada 0.5
d_biaffine 1024
d_char_emb 64
d_ff 2048
d_kv 64
d_label_hidden 250
d_model 1024
dataset 'ptb'
elmo_dropout 0.5
embedding_dropout 0.2
embedding_path 'data/word2vec_vi_words_100dims.txt.gz'
embedding_type 'sskip'
learning_rate 0.0008
learning_rate_warmup_steps 40
max_len_dev 0
max_len_train 0
model_name 'joint_single'
morpho_emb_dropout 0.2
num_heads 8
num_layers 12
pad_left False
partitioned True
punctuation ['.', '`', "'", ':', ',', '"', '...', '/', ';', '-']
relu_dropout 0.2
residual_dropout 0.2
sentence_max_len 300
step_decay True
step_decay_factor 0.5
step_decay_patience 5
tag_emb_dropout 0.2
timing_dropout 0.0
use_bert False
use_bert_only False
use_cat True
use_chars_lstm True
use_elmo False
use_tags True
use_words True
use_xlnet False
word_emb_dropout 0.4
xlnet_do_lower

In [ ]:
#test15 with data16(dep data is DatQuocNguyen data)
!sh run_single.sh

In [ ]:
!sh test.sh

/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/.shortcut-targets-by-id/1DP82lkt284bhnm7X9kA_LML9IgUxQIL9/Khoa_luan_tot_nghiep/HPSG-Parser/HPSG-Neural-Parser-master/src_joint/hpsg_decoder.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1969:0,
                 from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /root/.pyxbld/temp.linux-x86_64-3.7/pyrex/hpsg_decoder.c:698:
/usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!
